In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
import itertools
import math
import re
import lightgbm as lgb

In [3]:
idx = pd.IndexSlice


In [4]:
train =pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
transactions=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
holidays=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")

In [5]:
train['test'] = 0
test['test'] = 1

data = pd.concat([train, test], axis=0)

In [6]:
data = data.merge(holidays, on='date', how='left')
data= data.merge(stores, on='store_nbr', how='left')
data= data.merge(oil, on='date', how='left')
data= data.merge(transactions, on=['date', 'store_nbr'], how='left')
data = data.set_index(['store_nbr', 'date', 'family'])
data = data.drop(index='2013-01-01', level=1)
data

id   sales  onpromotion  \
store_nbr date       family                                                     
1         2013-01-02 AUTOMOTIVE                     1782     2.0            0   
                     BABY CARE                      1783     0.0            0   
                     BEAUTY                         1784     2.0            0   
                     BEVERAGES                      1785  1091.0            0   
                     BOOKS                          1786     0.0            0   
...                                                  ...     ...          ...   
9         2017-08-31 POULTRY                     3029395     NaN            1   
                     PREPARED FOODS              3029396     NaN            0   
                     PRODUCE                     3029397     NaN            1   
                     SCHOOL AND OFFICE SUPPLIES  3029398     NaN            9   
                     SEAFOOD                     3029399     NaN            0   

                                                 test type_x locale  \
store_nbr date       family                                           
1         2013-01-02 AUTOMOTIVE                     0    NaN    NaN   
                     BABY CARE                      0    NaN    NaN   
                     BEAUTY                         0    NaN    NaN   
                     BEVERAGES                      0    NaN    NaN   
                     BOOKS                          0    NaN    NaN   
...                                               ...    ...    ...   
9         2017-08-31 POULTRY                        1    NaN    NaN   
                     PREPARED FOODS                 1    NaN    NaN   
                     PRODUCE                        1    NaN    NaN   
                     SCHOOL AND OFFICE SUPPLIES     1    NaN    NaN   
                     SEAFOOD                        1    NaN    NaN   

                                                locale_name description  \
store_nbr date       family                                               
1         2013-01-02 AUTOMOTIVE                         NaN         NaN   
                     BABY CARE                          NaN         NaN   
                     BEAUTY                             NaN         NaN   
                     BEVERAGES                          NaN         NaN   
                     BOOKS                              NaN         NaN   
...                                                     ...         ...   
9         2017-08-31 POULTRY                            NaN         NaN   
                     PREPARED FOODS                     NaN         NaN   
                     PRODUCE                            NaN         NaN   
                     SCHOOL AND OFFICE SUPPLIES         NaN         NaN   
                     SEAFOOD                            NaN         NaN   

                                                transferred   city      state  \
store_nbr date       family                                                     
1         2013-01-02 AUTOMOTIVE                         NaN  Quito  Pichincha   
                     BABY CARE                          NaN  Quito  Pichincha   
                     BEAUTY                             NaN  Quito  Pichincha   
                     BEVERAGES                          NaN  Quito  Pichincha   
                     BOOKS                              NaN  Quito  Pichincha   
...                                                     ...    ...        ...   
9         2017-08-31 POULTRY                            NaN  Quito  Pichincha   
                     PREPARED FOODS                     NaN  Quito  Pichincha   
                     PRODUCE                            NaN  Quito  Pichincha   
                     SCHOOL AND OFFICE SUPPLIES         NaN  Quito  Pichincha   
                     SEAFOOD                            NaN  Quito  Pichincha   

                                              

In [7]:
data_ = data.copy().reset_index()

train = data_[data_['test'] == 0]
test = data_[data_['test'] == 1]

train['date'] = pd.to_datetime(train["date"])
train['day_of_week'] = train['date'].dt.day_of_week
train['day_of_year'] = train['date'].dt.dayofyear
train['day_of_month'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year

/tmp/ipykernel_55/1357458643.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['date'] = pd.to_datetime(train["date"])
/tmp/ipykernel_55/1357458643.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['day_of_week'] = train['date'].dt.day_of_week
/tmp/ipykernel_55/1357458643.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [8]:
grouping_columns = ['year', 'quarter', 'month', 'day_of_week', 'day_of_year', 'day_of_month']

for ind, column in enumerate(grouping_columns):
    grouped_data = train.groupby(column)['sales'].sum()
    grouped_data = pd.DataFrame(grouped_data).reset_index()

In [9]:
data_ = data.copy().reset_index()
grouped_data = data_.groupby(['store_nbr', 'family'])

In [10]:
alphas = [0.95, 0.8, 0.65, 0.5]
lags =[1,7,30]

for a in alphas:
    for i in lags:
        data_[f'sales_lag_{i}_alpha_{a}'] = np.log1p(grouped_data['sales'].transform(lambda x: x.shift(i).ewm(alpha=a, min_periods=1).mean()))

data_['sales_lag_7_alpha_0.5'].describe()

count    3.068604e+06
mean     3.002727e+00
std      2.665066e+00
min      0.000000e+00
25%      1.550843e-02
50%      2.562952e+00
75%      5.312741e+00
max      1.112374e+01
Name: sales_lag_7_alpha_0.5, dtype: float64

In [11]:
data_['date'] = pd.to_datetime(data_["date"])
data_['day_of_week'] = data_['date'].dt.day_of_week
data_['day_of_year'] = data_['date'].dt.dayofyear
data_['day_of_month'] = data_['date'].dt.day
data_['month'] = data_['date'].dt.month
data_['quarter'] = data_['date'].dt.quarter
data_['year'] = data_['date'].dt.year

In [12]:
data_['onpromotion'] = data_['onpromotion'].apply(lambda x: x > 0)


In [13]:
sales_lag_columns = list(data_.filter(like="lag").columns)


In [14]:
training_percentage = 0.8
testing_percentage = 0.2

to_dummies = ['day_of_week', 'day_of_month', 'month', 'quarter', 'year', 'store_nbr', 'type_y', 'cluster', 'family', 'onpromotion', 'type_x',
       'locale', 'locale_name', 'city', 'state']

X = data_.loc[:, [ 'day_of_week', 'day_of_month', 'month', 'quarter', 'year', 'store_nbr', 'type_y', 'cluster', 'family', 'onpromotion', 'type_x',
       'locale', 'locale_name',  'city', 'state', 'test', 'sales', 'id']+ sales_lag_columns]
X[to_dummies] = X[to_dummies].astype('category')

data_train = X[X['test'] == 0]
data_test = X[X['test'] == 1]

n = len(data_train)

training_start = 0
training_end = math.floor(n * training_percentage)
validation_start = training_end
validation_end = n

X_train = data_train.loc[training_start:training_end, :].drop(['test', 'sales', 'id'],  axis=1)
y_train = data_train.loc[training_start:training_end, 'sales']
X_val = data_train.loc[validation_start:validation_end,  :].drop(['test', 'sales', 'id'],  axis=1)
y_val = data_train.loc[validation_start:validation_end, 'sales']

X_test = data_test.loc[:, ].drop(['test', 'sales', 'id'],  axis=1)

In [ ]:
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)) 
X_val = X_val.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)) 
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x)) 

In [ ]:
hyper_params = {'task': 'train','boosting_type': 'gbdt','objective': 'regression','metric': ['l1','l2'],'learning_rate': 0.1,
'feature_fraction': 0.9,'bagging_fraction': 0.7,'bagging_freq': 10,'verbose': 0,"max_depth": 50,"num_leaves": 128,"max_bin": 512}

gbm = lgb.LGBMRegressor(**hyper_params)

gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='l1')

In [ ]:
y_pred = gbm.predict(X_val)


In [ ]:
results = pd.concat([y_val.reset_index(drop=True), pd.Series(y_pred)], axis=1).rename(columns={'sales': 'y_val', 0: 'y_pred'})

results['y_pred'] = results['y_pred'].clip(0)

results = results[results['y_val'] > 10]

results

In [ ]:
y_pred = gbm.predict(X_test)
y_pred

In [ ]:
data_test['id']
output = pd.DataFrame(index=data_test['id'])
output['sales'] = y_pred
output['sales'] = output['sales'].clip(0)

In [ ]:
output.to_csv('/kaggle/working/submission.csv')